<a href="https://colab.research.google.com/github/skimaza/assist/blob/main/cnn_tl_dog_breed_classification_assist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI 전략경영MBA 경영자를 위한 딥러닝 원리의 이해
# Convolutional Neural Network 전이학습 실습 예제
# 전이학습을 이용한 견종 분류
- https://github.com/Ahmad-shaikh575/dog-breed-classifier 예제를 이용했음  
- 예제를 설명하는 글은 https://levelup.gitconnected.com/dog-breed-classifier-with-pytorch-using-transfer-learning-8f15af6f9010

# 실행전에 Colab 메뉴에서 런타임->런타임 유형변경을 선택하여 하드웨어 가속기가 GPU로 선택되었는지 확인
### GPU를 사용하지 않아도 실행할 수 있으나 학습에 시간이 오래 걸림

# 학습에 사용할 데이터셋 다운로드

- urllib 라이브러리의 request 모듈의 urlretrieve 함수를 이용 (https://docs.python.org/3/library/urllib.request.html)  
- 첫번째 인자는 URL, 두번째 인자는 저장할 파일명  
- (주의) urlretrieve를 사용할 때 두번째 인자를 지정하지 않으면 임시 디렉토리(/tmp)에 임의의 이름으로 저장됨.

In [ ]:
import os
import urllib.request
urllib.request.urlretrieve('https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip', 'dogImages.zip')

## 아래 코드는 다운로드가 안 되는 경우를 위한 코드
### 라인 앞의 '#'을 지우고 실행
- (주의) 이번 파일은 크기가 커서 MLP 예제와 다르게 wget 명령문에서 복잡한 처리를 해줘야 다운로드가 성공함
- 다운로드후 '!ls -l' 명령 결과 dogImages.zip 파일크기가 1132023110 바이트인지 꼭 확인


In [ ]:
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JQ-A69KBwYlS1yFNP2Yz2K5dwysgcxcB' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JQ-A69KBwYlS1yFNP2Yz2K5dwysgcxcB" -O dogImages.zip && rm -rf /tmp/cookies.txt

# 데이터 살펴보기

dogImages.zip이 다운로드됐는지 확인

In [ ]:
!ls -l

압축 풀기

In [ ]:
!unzip 'dogImages.zip' > /dev/null  # 화면 출력 메시지를 표시하지 않기 위해 뒤에 "> /dev/null"을 붙였음. 

In [ ]:
!ls dogImages

이미 train, valid, test 데이터셋이 구분되어 있음

In [ ]:
!ls dogImages/train

### 133 종류의 개 이미지가 디렉토리별로 저장되어 있다

In [ ]:
!ls dogImages/train/001.Affenpinscher/

In [ ]:
!ls dogImages/train/133.Yorkshire_terrier/

In [ ]:
!ls dogImages/valid/001.Affenpinscher/

In [ ]:
!ls dogImages/test/001.Affenpinscher/

### 데이터셋을 본 후 간단한 요약
- 데이터는 train, valid, test 디렉토리로 구분되어 있음
- 각 디렉토리에는 133개 견종 디렉토리가 'ddd.breed' 형식으로 구성되어 있음
- 견종 디렉토리에는 'breed_ddddd.jpg' 형식의 이름을 갖는 jpg 이미지 파일들이 있음
- 견종 디렉토리에 있는 이미지의 개수는 견종별로 일치하지 않는 것 같음

### 전체 데이터에 대한 분포를 확인하여 위 요약을 검증

glob는 디렉토리에서 파일리스트를 읽어오는 라이브러리  
읽어온 결과의 순서는 이름순이 아님

In [ ]:
import numpy as np
from glob import glob

In [ ]:
datadir = 'dogImages'
train_breed_dirs = glob(datadir + '/train/*')
valid_breed_dirs = glob(datadir + '/valid/*')
test_breed_dirs = glob(datadir + '/test/*')

학습데이터셋 크기와 처음 10가지 종류 보기

In [ ]:
print(len(train_breed_dirs))
print(train_breed_dirs[:10])

In [ ]:
len(valid_breed_dirs), len(test_breed_dirs)

순서가 섞여서 보기 어려움  
natsort 패키지를 이용해서 사람이 보는 방식으로 정렬

In [ ]:
from natsort import natsorted

In [ ]:
train_breed_dirs = natsorted(train_breed_dirs)
valid_breed_dirs = natsorted(valid_breed_dirs)
test_breed_dirs = natsorted(test_breed_dirs)
train_breed_dirs[:10]

정렬이 되었음

### train_breed_dirs 안에 있는 파일 개수 확인

plotting을 위해 names에 디렉토리, train_counts에 디렉토리의 이미지 개수를 저장

In [ ]:
names = []
train_counts = []
for d in train_breed_dirs:
    breed_name = d.split('/')[-1]
    image_count = len(glob(d + '/*.jpg'))
    names.append(breed_name)
    train_counts.append(image_count)
    print("{:40}: {} images for train".format(breed_name, image_count))

names 제일 끝 3개의 요소와 데이터수 확인

In [ ]:
names[-3:]

In [ ]:
train_counts[-3:]

# plotting을 위해 pyplot 임포트

In [ ]:
from matplotlib import pyplot as plt

그림 크기를 키우기 위해 디폴트 설정을 변경

In [ ]:
plt.rcParams['figure.figsize'] = (20, 10)

클래스별(디렉토리별)로 train data 개수를 플로팅

In [ ]:
plt.plot(names, train_counts)
plt.xticks(rotation=90, fontsize=10);    # 여기에 세미콜론을 넣어서 주피터에서 xticks의 리턴값을 보여주지 않도록 지정했다. 

valid와 test data에 대해서 동일한 플로팅

In [ ]:
valid_counts = []
for d in valid_breed_dirs:
    breed_name = d.split('/')[-1]
    image_count = len(glob(d + '/*.jpg'))
    # names.append(breed_name)  # names는 train에서 이미 구했으므로 여기서 다시 모을 필요없음
    valid_counts.append(image_count)
    print("{:40}: {} images for validation".format(breed_name, image_count))

In [ ]:
plt.plot(names, valid_counts)
plt.xticks(rotation=90, fontsize=10);

In [ ]:
test_counts = []
for d in test_breed_dirs:
    breed_name = d.split('/')[-1]
    image_count = len(glob(d + '/*.jpg'))
    test_counts.append(image_count)
    print("{:40}: {} images for test".format(breed_name, image_count))

In [ ]:
plt.plot(names, test_counts)
plt.xticks(rotation=90, fontsize=10);

train, valid, test count를 한꺼번에 플로팅하여 데이터수를 상대적으로 비교

pyplot은 한 셀에서는 plt.show()를 실행하거나 셀이 끝나기 전까지 같은 그래프에 그려준다.  
셀이 나눠지면 다른 그림이 나오게 되므로 아래 명령은 한 셀에서 실행해야 한다

In [ ]:
plt.plot(names, train_counts, label='train')
plt.plot(names, valid_counts, label='valid')
plt.plot(names, test_counts, label='test')
plt.legend(fontsize=18)
plt.xticks(rotation=90, fontsize=10)
plt.show() # 같은 셀에서 실행하고 있으므로 이 명령은 필요없다. 여러 데이터를 한 그래프에 표시하는 기능을 명확히 표시하기 위해 넣어두었음

train은 클래스당 35-75개, valid와 test는 클래스당 5-10개 수준임을 확인할 수 있다

# 이미지를 직접 확인하자

## PIL(Python Image Library)은 파이썬에서 이미지를 처리하는 라이브러리 중 하나

In [ ]:
from PIL import Image

In [ ]:
train_breed_dirs[0]

In [ ]:
glob(train_breed_dirs[0] + '*/*.jpg')[:10]

train_breed_dirs의 첫번째 디렉토리(dogImages/train/001.Affenpinscher)에서 모든 .jpg 파일을 읽고 그 중 첫번째 파일을 읽어들인다

In [ ]:
filename = glob(train_breed_dirs[0] + '/*.jpg')[0]
print(filename)
tr_affen_img = Image.open(filename)
plt.imshow(np.asarray(tr_affen_img));

### 두 종류에 대해서 이미지 보기

In [ ]:
for d in train_breed_dirs[:2]:
    filename = glob(d + '/*.jpg')[0]
    img = Image.open(filename)
    plt.imshow(np.asarray(img))
    print(filename)
    print('디렉토리', d.split('/')[-1])
    plt.show(); # 그림을 여러개 그리기 때문에 매번 plt.show()를 해서 화면에 표시한다. 주피터 노트북 각 셀에서는 마지막에 자동으로 show를 처리해 주기 때문에 앞선 예제에서는 이 명령이 생략되었었다

## 여러 클래스 확인

계산에 사용할 파이썬 내장 나머지와 몫 연산 기능 확인  
파이썬 //(나눗셈 몫), %(나머지)

In [ ]:
print('0/10의 몫과 나머지', 0 // 10, 0 % 10)
print('132/10의 몫과 나머지', 132 // 10, 132 % 10)
print('139/10의 나머지', 139//10)

40개만 그려보자

In [ ]:
fig, axes = plt.subplots(4, 10, figsize=(30,15)) # 4 rows, 10 columns 형태로 여러 이미지 그리기
for ind, d in enumerate(train_breed_dirs[:40]):
    breed_idx_name = d.split('/')[-1]
    img = np.asarray(Image.open(glob(d + '/*.jpg')[0]))
    row = ind // 10 # 한 줄에 10개씩 그리기
    col = ind % 10 # 한 줄 안에서 순서
    axes[row, col].imshow(img)
    axes[row, col].set_title(breed_idx_name)
plt.tight_layout()
plt.show()
plt.close(fig) # subplots의 fig를 close 하지 않으면 메모리가 반환되지 않아서 가용메모리가 점차 줄어든다.

In [ ]:

# load filenames for human and dog images
dog_files = np.array(glob("/content/dogImages/*/*/*"))
# print number of images in each dataset
print('There are %d total dog images.' % len(dog_files))

# Pretrained model download
전이학습을 위해 이미지넷으로 학습된 모델을 다운로드  
예제에서는 VGG11 CNN 모델을 사용

In [ ]:
import torch
import torchvision.models as models

# define VGG11 model
VGG11 = models.vgg11(pretrained=True)

# check if CUDA is available
# Colab 시작할 때 런타임 유형을 GPU로 선택했으면 torch.cuda.is_available()가 True
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG11 = VGG11.cuda()

In [ ]:
use_cuda

VGG11은 2차원 CONV 8개, Fully Connected 3개로 구성된 11 레이어 CNN 모델  
레이어의 채널수
- Conv 레이어 3(입력 RGB 3ch)-64-128-256-256-512-512-512-512 Conv
- FC 레이어 4096-4096-1000(출력 1000 클래스) FC 레이어


- 3,4번 Conv, 5,6번 Conv, 7,8번 Conv 사이에는 MaxPooling을 하지 않아서 feature map 크기가 유지되고 있음.

In [ ]:
VGG11

마지막 classifier 변수에 두 개의 fully connected layer와 output layer가 있다.  
output layer는 classifier[6] 이다.

# 데이터 로더 구성

In [ ]:
from torchvision import datasets
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torchvision.transforms as transforms


In [ ]:
data_dir = '/content/dogImages'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

## 이번에는 단순히 ToTensor뿐 아니라 이미지에 대해 rotation, resizeCrop, HorizontalFlip, Normalize까지 적용하여 data augmentation 한 후에 학습 진행
### 이미지 데이터 augmentation 결과 보기

In [ ]:
data_augmentation_transforms = transforms.Compose([transforms.RandomRotation(30),
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor(),
                                                    #transforms.Normalize([0.485, 0.456, 0.406], 
                                                    #                     [0.229, 0.224, 0.225])
                                                    ])
augmented_data = datasets.ImageFolder(train_dir, transform=data_augmentation_transforms)
augdata_loader = torch.utils.data.DataLoader(augmented_data, batch_size=20, shuffle=True)

# 미니배치 하나에 대해 augmented data 보기
for b, l in augdata_loader:
    for i in range(len(b)):
        print('Image shape:', list(b[i].shape), 'Label:', l[i].numpy(), 'Name:', names[l[i]])
        img = b[i].numpy() # tensor를 numpy array로 변환
        img = np.transpose(img, (1, 2, 0)) # color 채널 순서 맞춰주기
        plt.imshow(img)
        plt.show()
    break

# 정규화를 적용한 이미지 보기
## 정규화는 pretrained model 원본 데이터셋(이미지넷)의 분포에서 계산된 값
## torchvision.models 의 모든 모델은 mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] 으로 정규화된 입력을 받도록 학습되어 있음
## 참고 링크
https://pytorch.org/vision/stable/models.html   
https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2 

In [ ]:
data_augmentation_transforms_normalized = transforms.Compose([transforms.RandomRotation(30),
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.RandomHorizontalFlip(),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                                         [0.229, 0.224, 0.225])
                                                    ])
augmented_data_normalized = datasets.ImageFolder(train_dir, transform=data_augmentation_transforms_normalized)
augdata_loader_normalized = torch.utils.data.DataLoader(augmented_data_normalized, batch_size=20, shuffle=True)

# 정규화된 이미지 보기
for b, l in augdata_loader_normalized:
    for i in range(5):
        print('Image shape:', list(b[i].shape), 'Label:', l[i].numpy(), 'Name:', names[l[i]])
        img = b[i].numpy() # tensor를 numpy array로 변환
        img = np.transpose(img, (1, 2, 0)) # color 채널 순서 맞춰주기
        plt.imshow(img)
        plt.show()
    break

## 전체 데이터로더 구성
데이터 augmentation에 데이터 값의 정규화까지 적용.  
이미지넷 데이터셋의 정규화 파라미터 적용: [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

In [ ]:
data_dir = '/content/dogImages'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])
                                       ])


valid_transforms = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])
# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir,transform=valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=20,shuffle=False)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=20,shuffle=False)
loaders_transfer = {'train':trainloader,
                  'valid':validloader,
                  'test':testloader}
data_transfer = {
    'train':trainloader
}

# vgg11 모델

In [ ]:
import torchvision.models as models
import torch.nn as nn
model_transfer = models.vgg11(pretrained=True)

for param in model_transfer.parameters():
    param.requires_grad = False

if use_cuda:
    model_transfer = model_transfer.cuda()
print(model_transfer)

# pretrained model의 모든 weight를 고정했음 (requires_grad 가 False) 
# 마지막 출력 레이어인 classifier[6]도 고정된 상태

In [ ]:
model_transfer.classifier[3].weight.requires_grad

In [ ]:
model_transfer.classifier[6], model_transfer.classifier[6].weight.requires_grad

# 출력 레이어가 이미지넷에 맞춰 1000개의 클래스로 되어 있다.
# 이 구조를 자신의 문제에 맞춰 바꿔 준다.
# dog breed가 133 종류이니 133개의 출력 레이어로 바꾼다

In [ ]:
model_transfer.classifier[6] = nn.Linear(4096,133,bias=True) # 출력 레이어를 133 출력 노드를 가지는 층으로 변경

In [ ]:
model_transfer

# 새로 정의한 레이어는 디폴트로 requires_grad가 True임. 즉 학습할 수 있는 레이어

In [ ]:
model_transfer.classifier[6], model_transfer.classifier[6].weight.requires_grad

# Loss 함수와 Optimizer 정의

In [ ]:
import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.SGD(model_transfer.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        # 학습 모드로 설정
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']): # 매번 loader에서 하나의 미니 배치를 읽는다
            # GPU를 사용하고 있으면 데이터를 cuda 형식으로 변환
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                model.to('cuda')
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like

            # 그래디언트를 0으로 초기화
            optimizer.zero_grad()
            # 모델의 forward pass를 실행하여 출력을 계산
            output = model(data)
            # 출력과 target 사이의 loss 계산
            loss = criterion(output,target)
            # backward pass를 실행하여 그래디언트를 계산
            loss.backward()
            # weight update
            optimizer.step()
            # train_loss를 계산
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))

        ######################    
        # validate the model #
        ######################
        # evaluation 모드로 설정 - weight가 update되지 않는다
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            accuracy=0
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            logps = model(data)
            loss = criterion(logps, target)
            # 학습이 아니므로 backward pass가 없다

            # validation loss 계산
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model


## 20 epoch 학습 (or 10 epochs)

In [ ]:
# train the model
#model_transfer = train(10, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')
model_transfer = train(20, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

# 모델의 prediction과 target 을 비교하여 accuracy를 계산

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    


In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

In [ ]:
!ls dogImages/test

In [ ]:
!mkdir my_dogs

In [ ]:
!cp dogImages/test/*/*.jpg my_dogs

In [ ]:
!ls my_dogs

In [ ]:
dog_files_short = np.array(glob("/content/my_dogs/*"))
# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].dataset.classes]

model_transfer.load_state_dict(torch.load('model_transfer.pt'))


In [ ]:
# predict function
def predict_breed_transfer(img_path, model, classnames):
    # load the image and return the predicted breed
    
    transform = transforms.Compose([transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])])
    img = Image.open(img_path)
    img = transform(img)[:3,:,:].unsqueeze(0)
    if use_cuda:
        img = img.cuda()
        model.to('cuda')
    # forward pass: compute predicted outputs by passing inputs to the model
    model.eval()
    idx = torch.argmax(model(img))
    return class_names[idx]


In [ ]:
plt.rcParams['figure.figsize'] = (5, 5)

In [ ]:
len(dog_files_short)

In [ ]:
all_count = 0
correct_count = 0
missed_count = 0
for fn in dog_files_short[:100]:
    img = Image.open(fn)
    label = fn.split('/')[-1].split('.')[0][:-6].replace('_', ' ')
    print('Ground truth', label)
    plt.imshow(np.asarray(img))
    plt.show()
    #pred = predict_breed_transfer(fn)
    pred = predict_breed_transfer(fn, model_transfer, class_names)
    print('Prediction', pred)
    all_count += 1
    if label == pred:
        print('CORRECT')
        correct_count += 1
    else:
        print('MISSED')
        missed_count += 1
    print('------------------------------------------------------------')
    #break

print("Correct results: {:4.2%} ({} out of {})".format(correct_count/all_count, correct_count, all_count))


# 보다 큰 모델로 전이학습

In [ ]:
#model_transfer_resnet = models.resnet50(pretrained=True)
model_transfer_resnext = models.resnext50_32x4d(pretrained=True)

In [ ]:
#model_transfer_resnet
model_transfer_resnext


In [ ]:
#model_transfer_resnet.fc
model_transfer_resnext.fc

# ResNet50는 출력레이어 이름이 fc이고 2048x1000 구조임 (resnext50도 동일)
## 출력레이어를 2048x133으로 바꿔주고 전이학습 실행

# 아래 코드를 그대로 사용하기 위해 이름을 model_transfer_resnet으로 변경

In [ ]:
model_transfer_resnet = model_transfer_resnext

In [ ]:
for param in model_transfer_resnet.parameters():
    param.requires_grad = False

if use_cuda:
    model_transfer_resnet = model_transfer_resnet.cuda()

# 출력 레이어를 133 출력 노드를 가지는 층으로 변경
model_transfer_resnet.fc = nn.Linear(2048, 133, bias=True) 

# 변경 상태 확인
print(model_transfer_resnet)
print(model_transfer_resnet.fc.weight.requires_grad)


# Loss criterion과 optimizer 설정

In [ ]:
criterion_transfer_resnet = nn.CrossEntropyLoss()
optimizer_transfer_resnet = optim.SGD(model_transfer_resnet.parameters(), lr=0.001, momentum=0.9)

# train 20 epochs

In [ ]:
# train the model
model_transfer_resnet = train(20, loaders_transfer, model_transfer_resnet, optimizer_transfer_resnet, criterion_transfer_resnet, use_cuda, 'model_transfer_resnet.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer_resnet.load_state_dict(torch.load('model_transfer_resnet.pt'))

# test

In [ ]:
test(loaders_transfer, model_transfer_resnet, criterion_transfer_resnet, use_cuda)

# check prediction accuracy

In [ ]:
# predict function
# def predict_breed_transfer(img_path, model, classnames):
#     # load the image and return the predicted breed
    
#     transform = transforms.Compose([transforms.Resize(255),
#                                     transforms.CenterCrop(224),
#                                     transforms.ToTensor(),
#                                     transforms.Normalize([0.485, 0.456, 0.406], 
#                                                          [0.229, 0.224, 0.225])])
#     img = Image.open(img_path)
#     img = transform(img)[:3,:,:].unsqueeze(0)
#     if use_cuda:
#         img = img.cuda()
#         model.to('cuda')
#     # forward pass: compute predicted outputs by passing inputs to the model
#     model.eval()
#     idx = torch.argmax(model(img))
#     return class_names[idx]


In [ ]:

dog_files_short = np.array(glob("/content/my_dogs/*"))
# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].dataset.classes]

model_transfer_resnet.load_state_dict(torch.load('model_transfer_resnet.pt'))

all_count = 0
correct_count = 0
missed_count = 0
for i, fn in enumerate(dog_files_short[:100]):
    img = Image.open(fn)
    label = fn.split('/')[-1].split('.')[0][:-6].replace('_', ' ')
    print(i)
    print('Ground truth', label)
    plt.imshow(np.asarray(img))
    plt.show()
    pred = predict_breed_transfer(fn, model_transfer_resnet, class_names)
    print('Prediction', pred)
    all_count += 1
    if label == pred:
        print('CORRECT')
        correct_count += 1
    else:
        print('MISSED')
        missed_count += 1
    print()
    #break

print("Correct results: {:4.2%} ({} out of {})".format(correct_count/all_count, correct_count, all_count))


# 끝